### Cine21 영화배우 정보 웹스크래핑
1. Site 주소 : http://www.cine21.com/rank/person/
2. Request URL : http://www.cine21.com/rank/person/content   
  - 크롬 개발자 도구의 network 탭에서 xhr 인 것만 필터링 
3. Request Method : POST
4. Form Data 
 - section: actor
 - period_start: 2020-09
 - gender: all
 - page: 1
5. 2년치 데이터를 한꺼번에 스크래핑 해서 actors.json 파일로 저장 

In [2]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin


### 1단계 
* 1 page 7명의 이름과 상세정보 추출

### 정규표현식 (regular expression)
* \w는 word를 표현하며 알파벳 + 숫자 + _ 중의 한 문자임을 의미한다.
* 문자가 출현한 횟수를 나타낼때 
 <pre>   
  * : 0 ~ n,  + : 1 ~ n , ? : 0(zero) or 1 
 </pre> 
* 정규표현식에 특수문자를 사용할때는 특수문자 앞에 \(back slash)를 반드시 주어야 한다
* .(period)는 줄바꿈 문자(\n)을 제외한 모든 글자 1개를 의미한다
* Greedy vs Non-Greedy
 - Greedy 욕심많은 (.*) : 모든 태그가 다 선택
 - Non-Greedy 욕심없는 (.*?) : 첫번째 매칭되는 태그만 선택
* [\n\r\t] : \n,\r,\t 중에 하나만 찾는다.
 - re.compile('[\n\r\t]')는 해당 정규표현식을 찾아주는 컴파일러를 생성한다.
 - compile 함수내의 정규표현식 앞에는 항상 r 을 붙혀 주어야 한다.

In [3]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2020-09',
    'gender': 'all',
    'page': 1
}
res = requests.post(req_url, data=req_param)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    
    for li_tag in soup.select('li.people_li'):
        #print(li_tag.select('div.name'))
        div_tag = li_tag.select_one('div.name')
        detail_url = div_tag.select_one('a')['href']
        detail_url = urljoin(req_url,detail_url)
        name = div_tag.text
        result = re.sub('\(\w*\)','',name)
        #print(result, detail_url)
                       
        res = requests.get(detail_url)
        if res.ok:
            soup = BeautifulSoup(res.text,'html.parser')
            for li_tag in soup.select('ul.default_info li'):
                actor_dict_key = li_tag.select_one('span.tit').text
                #print(li_tag)
                #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                #<li>배우</li> => 배우
                actor_dict_value = re.sub('<.*?>','',result)
                
                regex = re.compile(r'[\n\r\t]')
                actor_dict_value = regex.sub('',actor_dict_value)        
                print(actor_dict_key,actor_dict_value)
                #print(li_tag)

200
직업 배우
생년월일 1967-04-27
성별 남
신장/체중 177cm, 78kg
학교 유한대학 기계설계학
직업 배우
생년월일 1978-06-28
성별 여
홈페이지 https://twitter.com/hajiwon1023https://instagram.com/hajiwon1023
신장/체중 168cm, 45kg
학교 단국대학교 연극영화학 학사
취미 만화책보기, 소품만들기, 청소하기
특기 검도, 골프, 승마, 재즈댄스, 장구춤
직업 배우
생년월일 1971-01-10
성별 남
신장/체중 180cm
학교 서울예술대학 연극과
소속사 DMS커뮤니케이션즈
다른 이름 엄홍식
직업 배우
생년월일 1986-10-06
성별 남
홈페이지 https://www.instagram.com/hongsick/https://twitter.com/seeksik
신장/체중 178cm, 60kg
학교 단국대학교 연극
취미 피아노 연주, 인터넷 게임, 영화, 음악감상
특기 스노우보드, 수영
직업 배우
생년월일 2012-03-12
성별 여
직업 배우
생년월일 1973-06-03
성별 남
직업 배우
생년월일 1941-11-30
성별 여
홈페이지 http://www.namuni.kr/
학교 창덕여자고등학교


### 2단계 
* 1 page 7명의 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.

In [4]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2020-09',
    'gender': 'all',
    'page': 1
}
res = requests.post(req_url, data=req_param)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    
    actor_detail_list = []
    actor_li_tag = soup.select('li.people_li')    
    
    for li_tag in actor_li_tag:
        actor_detail_dict = {}
        
        #print(li_tag.select('div.name'))
        div_tag = li_tag.select_one('div.name')
        #상세페이지 url 추출
        detail_url = div_tag.select_one('a')['href']
        detail_url = urljoin(req_url,detail_url)
        
        #배우이름 추출
        name = div_tag.text
        #성동일(1편) => 성동일 
        actor_name = re.sub('\(\w*\)','',name)
        actor_detail_dict['배우이름'] = actor_name
        
        #순위(ranking) 추출
        #print(li_tag.select_one('span.grade'))                   
        actor_ranking = li_tag.select_one('span.grade').text
        actor_detail_dict['랭킹'] = int(actor_ranking)
        
        #흥행지수 추출
        actor_hit = li_tag.select_one('ul.num_info > li > strong').text
        actor_detail_dict['흥행지수'] = int(actor_hit.replace(',',''))
        
        #상세정보 페이지에 요청을 보냄    
        res = requests.get(detail_url)
        if res.ok:
            soup = BeautifulSoup(res.text,'html.parser')
            
            #기본정보
            for li_tag in soup.select('ul.default_info li'):
                actor_dict_key = li_tag.select_one('span.tit').text
                #print(li_tag)
                #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                #<li>배우</li> => 배우
                actor_dict_value = re.sub('<.*?>','',result)
                
                regex = re.compile(r'[\n\r\t]')
                actor_dict_value = regex.sub('',actor_dict_value)        
                #print(actor_dict_key,actor_dict_value)
                
                #상세페이지에 추출한 key와 value를 dict에 저장
                actor_detail_dict[actor_dict_key] = actor_dict_value
            
            #참여작품
            movie_title_list = []
            for movie_li_tag in soup.select('ul.part_works li'):
                movie_title = movie_li_tag.select_one('span.tit').text
                movie_title_list.append(movie_title)
            
            #print(movie_title_list)
            actor_detail_dict['출연영화'] = movie_title_list
            
            #상세정보 담은 dict를 list에 추가
            actor_detail_list.append(actor_detail_dict)

print(actor_detail_list[0:2])            

200
[{'배우이름': '성동일', '랭킹': 1, '흥행지수': 137320, '직업': '배우', '생년월일': '1967-04-27', '성별': '남', '신장/체중': '177cm, 78kg', '학교': '유한대학 기계설계학', '출연영화': ['담보', '방법', '어쩌다 가족']}, {'배우이름': '하지원', '랭킹': 2, '흥행지수': 102990, '직업': '배우', '생년월일': '1978-06-28', '성별': '여', '홈페이지': 'https://twitter.com/hajiwon1023https://instagram.com/hajiwon1023', '신장/체중': '168cm, 45kg', '학교': '단국대학교 연극영화학 학사', '취미': '만화책보기, 소품만들기, 청소하기', '특기': '검도, 골프, 승마, 재즈댄스, 장구춤', '출연영화': ['담보', '초콜릿', '갈릴레오 : 깨어난 우주']}]


### 3단계 
* 1,2,3 page 21명의 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.

In [5]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2020-09',
    'gender': 'all'
}

#21명의 배우정보 dict를 저장할 리스트
actor_detail_list = []

for page in range(1,4):
    req_param['page'] = page
    res = requests.post(req_url, data=req_param)
    #print(res.status_code)
    print(f'---> {page} 페이지 출력됨')
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')

        actor_li_tag = soup.select('li.people_li')
        for idx,li_tag in enumerate(actor_li_tag,1):
            actor_detail_dict = {}

            #print(li_tag.select('div.name'))
            div_tag = li_tag.select_one('div.name')
            #상세페이지 url 추출
            detail_url = div_tag.select_one('a')['href']
            detail_url = urljoin(req_url,detail_url)

            #배우이름 추출
            name = div_tag.text
            #성동일(1편) => 성동일 
            actor_name = re.sub('\(\w*\)','',name)
            actor_detail_dict['배우이름'] = actor_name
            
            print(f'======> {idx} : {actor_name}')

            #순위(ranking) 추출
            #print(li_tag.select_one('span.grade'))                   
            actor_ranking = li_tag.select_one('span.grade').text
            actor_detail_dict['랭킹'] = int(actor_ranking)

            #흥행지수 추출
            actor_hit = li_tag.select_one('ul.num_info > li > strong').text
            actor_detail_dict['흥행지수'] = int(actor_hit.replace(',',''))

            #상세정보 페이지에 요청을 보냄    
            res = requests.get(detail_url)
            if res.ok:
                soup = BeautifulSoup(res.text,'html.parser')

                #기본정보
                for li_tag in soup.select('ul.default_info li'):
                    actor_dict_key = li_tag.select_one('span.tit').text
                    #print(li_tag)
                    #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                    result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                    #<li>배우</li> => 배우
                    actor_dict_value = re.sub('<.*?>','',result)

                    regex = re.compile(r'[\n\r\t]')
                    actor_dict_value = regex.sub('',actor_dict_value)        
                    #print(actor_dict_key,actor_dict_value)

                    #상세페이지에 추출한 key와 value를 dict에 저장
                    actor_detail_dict[actor_dict_key] = actor_dict_value

                #참여작품
                movie_title_list = []
                for movie_li_tag in soup.select('ul.part_works li'):
                    movie_title = movie_li_tag.select_one('span.tit').text
                    movie_title_list.append(movie_title)

                #print(movie_title_list)
                actor_detail_dict['출연영화'] = movie_title_list

                #상세정보 담은 dict를 list에 추가
                actor_detail_list.append(actor_detail_dict)

print(len(actor_detail_list))                

---> 1 페이지 출력됨
======> 1 : 성동일
======> 2 : 하지원
======> 3 : 김희원
======> 4 : 유아인
======> 5 : 박소이
======> 6 : 유재명
======> 7 : 나문희
---> 2 페이지 출력됨
======> 1 : 이희준
======> 2 : 김호중
======> 3 : 고아성
======> 4 : 이솜
======> 5 : 박혜수
======> 6 : 최원영
======> 7 : 조현철
---> 3 페이지 출력됨
======> 1 : 곽도원
======> 2 : 김대명
======> 3 : 문승아
======> 4 : 김종수
======> 5 : 김원해
======> 6 : 김상호
======> 7 : 신민아
21


### 4단계 
* 모든 page의 배우 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.
* itertools 모듈의 count() 함수를 사용하여 무한 Looping
 - 무한 Loop를 탈출할 수 있는 조건
 - actor_li_tag = soup.select('li.people_li') 의 actor_list_tag의 길이가 0 이면 탈출

In [6]:
from itertools import count

req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2018-10',
    'gender': 'all'
}

#21명의 배우정보 dict를 저장할 리스트
actor_detail_list = []

for page in count(1):
    req_param['page'] = page
    res = requests.post(req_url, data=req_param)
    #print(res.status_code)
    if page % 50 == 0:
        print(f'---> {page} 페이지 출력됨')
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')

        actor_li_tag = soup.select('li.people_li')
        
        #해당 page가 없으면 loop를 탈출
        if len(actor_li_tag) == 0:
            break
            
        for idx,li_tag in enumerate(actor_li_tag,1):
            actor_detail_dict = {}

            #print(li_tag.select('div.name'))
            div_tag = li_tag.select_one('div.name')
            #상세페이지 url 추출
            detail_url = div_tag.select_one('a')['href']
            detail_url = urljoin(req_url,detail_url)

            #배우이름 추출
            name = div_tag.text
            #성동일(1편) => 성동일 
            actor_name = re.sub('\(\w*\)','',name)
            actor_detail_dict['배우이름'] = actor_name
            
            print(f'======> {idx} : {actor_name}')

            #순위(ranking) 추출
            #print(li_tag.select_one('span.grade'))                   
            actor_ranking = li_tag.select_one('span.grade').text
            actor_detail_dict['랭킹'] = int(actor_ranking)

            #흥행지수 추출
            actor_hit = li_tag.select_one('ul.num_info > li > strong').text
            actor_detail_dict['흥행지수'] = int(actor_hit.replace(',',''))

            #상세정보 페이지에 요청을 보냄    
            res = requests.get(detail_url)
            if res.ok:
                soup = BeautifulSoup(res.text,'html.parser')

                #기본정보
                for li_tag in soup.select('ul.default_info li'):
                    actor_dict_key = li_tag.select_one('span.tit').text
                    #print(li_tag)
                    #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                    result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                    #<li>배우</li> => 배우
                    actor_dict_value = re.sub('<.*?>','',result)

                    regex = re.compile(r'[\n\r\t]')
                    actor_dict_value = regex.sub('',actor_dict_value)        
                    #print(actor_dict_key,actor_dict_value)

                    #상세페이지에 추출한 key와 value를 dict에 저장
                    actor_detail_dict[actor_dict_key] = actor_dict_value

                #참여작품
                movie_title_list = []
                for movie_li_tag in soup.select('ul.part_works li'):
                    movie_title = movie_li_tag.select_one('span.tit').text
                    movie_title_list.append(movie_title)

                #print(movie_title_list)
                actor_detail_dict['출연영화'] = movie_title_list

                #상세정보 담은 dict를 list에 추가
                actor_detail_list.append(actor_detail_dict)

print(len(actor_detail_list))                        

======> 1 : 이병헌
======> 2 : 진선규
======> 3 : 하정우
======> 4 : 마동석
======> 5 : 이하늬
======> 6 : 류승룡
======> 7 : 유해진
======> 1 : 박정민
======> 2 : 황정민
======> 3 : 송강호
======> 4 : 조진웅
======> 5 : 신하균
======> 6 : 조우진
======> 7 : 이동휘
======> 1 : 이정재
======> 2 : 전혜진
======> 3 : 염정아
======> 4 : 공명
======> 5 : 장혜진
======> 6 : 이선균
======> 7 : 유아인
======> 1 : 조정석
======> 2 : 오정세
======> 3 : 이성민
======> 4 : 최우식
======> 5 : 박소담
======> 6 : 김윤석
======> 7 : 공효진
======> 1 : 김의성
======> 2 : 김아중
======> 3 : 김무열
======> 4 : 강기영
======> 5 : 곽도원
======> 6 : 성동일
======> 7 : 김종수
======> 1 : 배수지
======> 2 : 권해효
======> 3 : 임윤아
======> 4 : 주지훈
======> 5 : 이희준
======> 6 : 라미란
======> 7 : 이봉련
======> 1 : 고두심
======> 2 : 장기용
======> 3 : 권상우
======> 4 : 윤경호
======> 5 : 박인환
======> 6 : 최유화
======> 7 : 허준호
======> 1 : 김홍파
======> 2 : 조여정
======> 3 : 송영규
======> 4 : 박지환
======> 5 : 정해인
======> 6 : 강동원
======> 7 : 이서진
======> 1 : 정유미
======> 2 : 김지영
======> 3 : 류준열
======> 4 : 김희원
======> 5 : 김성오
======> 6 : 이정현
======> 7

======> 7 : 최윤영
======> 1 : 배누리
======> 2 : 백주희
======> 3 : 탁원정
======> 4 : 이학주
======> 5 : 이진주
======> 6 : 홍범기
======> 7 : 이주영
======> 1 : 이창민
======> 2 : 최명빈
======> 3 : 금새록
======> 4 : 장병관
======> 5 : 김해숙
======> 6 : 홍시호
======> 7 : 현봉식
======> 1 : 이일화
======> 2 : 최원형
======> 3 : 박영진
======> 4 : 진유영
======> 5 : 이찬원
======> 6 : 박중금
======> 7 : 이주원
======> 1 : 권혁범
======> 2 : 박영재
======> 3 : 박호산
======> 4 : 민경진
======> 5 : 이승연
======> 6 : 정의한
======> 7 : 이창훈
======> 1 : 방연지
======> 2 : 이한서
======> 3 : 조영진
======> 4 : 김보성
======> 5 : 손호준
======> 6 : 고수희
======> 7 : 박성광
======> 1 : 심완준
======> 2 : 연우진
======> 3 : 김보라
======> 4 : 정애화
======> 5 : 김승훈
======> 6 : 배정화
======> 7 : 김영광
======> 1 : 현경수
======> 2 : 정지훈
======> 3 : 최다니엘
======> 4 : 정종준
======> 5 : 강예원
======> 6 : 김창옥
======> 7 : 김호정
======> 1 : 임은정
======> 2 : 이소연
======> 3 : 윤송아
======> 4 : 조승우
======> 5 : 윤용식
======> 6 : 김옥경
======> 7 : 왕지혜
======> 1 : 김기두
======> 2 : 손선영
======> 3 : 이민혁
======> 4 : 이문식
======> 5 : 문남숙
======>

======> 3 : 주증녀
======> 4 : 조용주
======> 5 : 남유정
======> 6 : 이인성
======> 7 : 서태화
======> 1 : 황정순
======> 2 : 김영선
======> 3 : 김동영
======> 4 : 김민주
======> 5 : 김동원
======> 6 : 곽진무
======> 7 : 김현욱
======> 1 : 문희
======> 2 : 유다인
======> 3 : 유다미
======> 4 : 정상윤
======> 5 : 김재욱
======> 6 : 신석호
======> 7 : 이비안
======> 1 : 강길우
======> 2 : 최무룡
======> 3 : 김윤식
======> 4 : 남규리
======> 5 : 김조연
======> 6 : 김상원
======> 7 : 천정은
---> 150 페이지 출력됨
======> 1 : 정주원
======> 2 : 신소율
======> 3 : 김군자
======> 4 : 김다예
======> 5 : 김민주
======> 6 : 황유림
======> 7 : 곽규미
======> 1 : 강구한
======> 2 : 홍석연
======> 3 : 최수인
======> 4 : 윤승욱
======> 5 : 심은하
======> 6 : 이연
======> 7 : 이정현
======> 1 : 김도영
======> 2 : 이재원
======> 3 : 고원희
======> 4 : 류현경
======> 5 : 조미령
======> 6 : 이상각
======> 7 : 박노식
======> 1 : 동현배
======> 2 : 김정영
======> 3 : 차지현
======> 4 : 박상증
======> 5 : 조한선
======> 6 : 정영주
======> 7 : 




씨네21









서비스 이용에 불편을 드려 죄송합니다.
일시적으로 서비스 장애가 발생하였습니다.
빠른 시간안에 복구하도록 하겠습니다.
			





">




씨네21









서비스 이용에 불편

======> 1 : 양택조
======> 2 : 전계현
======> 3 : 이유미
======> 4 : 우돈기
======> 5 : 춘자
======> 6 : 안민영
======> 7 : 김재화
======> 1 : 김윤지
======> 2 : 정준하
======> 3 : 김기춘
======> 4 : 나잠 수
======> 5 : 이현행
======> 6 : 이연두
======> 7 : 소희정
======> 1 : 소희정
======> 2 : 스윙스
======> 3 : 이설
======> 4 : 류성록
======> 5 : 최지희
======> 6 : 박성우
======> 7 : 양은용
======> 1 : 박가영
======> 2 : 손병호
======> 3 : 송희준
======> 4 : 최석필
======> 5 : 공경은
======> 6 : 김동석
======> 7 : 문희라
======> 1 : 이승현
======> 2 : 채재일
======> 3 : 송금식
======> 4 : 강혜정
======> 5 : 홍수현
======> 6 : 고은아
======> 7 : 김애경
======> 1 : 장항선
======> 2 : 문용일
======> 3 : 김익태
======> 4 : 이용신
======> 5 : 이주일
======> 6 : 하명중
======> 7 : 김시후
======> 1 : 이용신
======> 2 : 진세연
======> 3 : 방수일
======> 4 : 이헌주
======> 5 : 최규환
======> 6 : 임선우
======> 7 : 




씨네21









서비스 이용에 불편을 드려 죄송합니다.
일시적으로 서비스 장애가 발생하였습니다.
빠른 시간안에 복구하도록 하겠습니다.
			





">




씨네21









서비스 이용에 불편을 드려 죄송합니다.
일시적으로 서비스 장애가 발생하였습니다.
빠른 시간안에 복구하도록 하겠습니다.
			






======> 1 : 정윤희
======> 2 : 전창

======> 3 : 연준석
======> 4 : 이영진
======> 5 : 김수경
======> 6 : 서윤아
======> 7 : 김신명
======> 1 : 연준석
======> 2 : 이영진
======> 3 : 김수경
======> 4 : 서윤아
======> 5 : 김신명
======> 6 : 권혁풍
======> 7 : 김최용준
======> 1 : 고경표
======> 2 : 박규채
======> 3 : 최민규
======> 4 : 이기영
======> 5 : 추귀정
======> 6 : 최제우
======> 7 : 염상태
======> 1 : 최민규
======> 2 : 이기영
======> 3 : 추귀정
======> 4 : 최제우
======> 5 : 염상태
======> 6 : 이종국
======> 7 : 홍경인
======> 1 : 하윤경
======> 2 : 서동수
======> 3 : 공지영
======> 4 : 노진원
======> 5 : 장성윤
======> 6 : 유대형
======> 7 : 현석
======> 1 : 노진원
======> 2 : 장성윤
======> 3 : 유대형
======> 4 : 현석
======> 5 : 이종국
======> 6 : 홍경인
======> 7 : 남태부
======> 1 : 김지영
======> 2 : 한주완
======> 3 : 구웅서
======> 4 : 이달형
======> 5 : 이재준
======> 6 : 주아름
======> 7 : 김정은
======> 1 : 김춘기
======> 2 : 나영희
======> 3 : 조향기
======> 4 : 이주연
======> 5 : 구웅서
======> 6 : 이달형
======> 7 : 이재준
======> 1 : 김영진
======> 2 : 조민아
======> 3 : 조향기
======> 4 : 김춘기
======> 5 : 나영희
======> 6 : 이주연
======> 7 : 구웅서
======> 1 : 이달형
======> 2

In [7]:
for actor in actor_detail_list:
    if len(actor['배우이름']) > 10:
        print(actor)

{'배우이름': '\n\n\n\n\n씨네21\n\n\n\n\n\n\n\n\n\n서비스 이용에 불편을 드려 죄송합니다.\n일시적으로 서비스 장애가 발생하였습니다.\n빠른 시간안에 복구하도록 하겠습니다.\n\t\t\t\n\n\n\n\n\n">\n\n\n\n\n씨네21\n\n\n\n\n\n\n\n\n\n서비스 이용에 불편을 드려 죄송합니다.\n일시적으로 서비스 장애가 발생하였습니다.\n빠른 시간안에 복구하도록 하겠습니다.\n\t\t\t\n\n\n\n\n\n', '랭킹': 356, '흥행지수': 416, '출연영화': ['웁스 더 어드벤쳐 컨티뉴스', '올 마이 라이프', '유코의 평형추']}
{'배우이름': '\n\n\n\n\n씨네21\n\n\n\n\n\n\n\n\n\n서비스 이용에 불편을 드려 죄송합니다.\n일시적으로 서비스 장애가 발생하였습니다.\n빠른 시간안에 복구하도록 하겠습니다.\n\t\t\t\n\n\n\n\n\n">\n\n\n\n\n씨네21\n\n\n\n\n\n\n\n\n\n서비스 이용에 불편을 드려 죄송합니다.\n일시적으로 서비스 장애가 발생하였습니다.\n빠른 시간안에 복구하도록 하겠습니다.\n\t\t\t\n\n\n\n\n\n', '랭킹': 1071, '흥행지수': 4, '출연영화': ['라이크 어 보스', '메모리 이레이저', '임프랙티컬 조커스: 더 무비']}
{'배우이름': '\n\n\n\n\n씨네21\n\n\n\n\n\n\n\n\n\n서비스 이용에 불편을 드려 죄송합니다.\n일시적으로 서비스 장애가 발생하였습니다.\n빠른 시간안에 복구하도록 하겠습니다.\n\t\t\t\n\n\n\n\n\n">\n\n\n\n\n씨네21\n\n\n\n\n\n\n\n\n\n서비스 이용에 불편을 드려 죄송합니다.\n일시적으로 서비스 장애가 발생하였습니다.\n빠른 시간안에 복구하도록 하겠습니다.\n\t\t\t\n\n\n\n\n\n', '랭킹': 1568, '흥행지수': 1, '출연영화': ['라이크 어 보스', '메모리 이레이저', '임프랙티컬 조커스: 더 무비']}
{'

In [8]:
print('before', len(actor_detail_list))
actor_detail_final_list = [actor for actor in actor_detail_list \
                           if len(actor['배우이름']) < 10]
print('after', len(actor_detail_final_list))

before 2459
after 2452


### 5단계
* actor_detail_final_list를 actors.json 파일로 저장하기

In [9]:
import json

with open('data/actors2.json','w',encoding='utf8') as file:
    json.dump(actor_detail_final_list, file)

print('json file write 종료')

json file write 종료


In [10]:
with open('data/actors2.json','r') as file:
    json_data = json.load(file)

print(len(json_data))    

2452


NameError: name 'col' is not defined